<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-research-and-practice/blob/main/nlp-for-vector-similarity-search/00_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TF-IDF

The respected grandfather of vector similarity search, born back in the 1970s. It consists of two parts, **Term Frequency (TF) and Inverse Document Frequency (IDF)**.

The TF component counts the number of times a term appears within a document and divides this by the total number of terms in that same document.

<img src='https://d33wubrfki0l68.cloudfront.net/d6e2ea146c9da6e32711b0a925f97d0027e9935c/83b7f/images/semantic-search-10.png' width='600'/>

>The term frequency (TF) component of TF-IDF counts the frequency of our query (‘bananas’) and divides by the frequency of all tokens.

That is the first half of our calculation, we have the frequency of our query within the current Document `f(q,D)` — over the frequency of all terms within the current Document `f(t,D)`.

The Term Frequency is a good measure, but doesn’t allow us to differentiate between common and uncommon words. If we were to search for the word ‘the’ — using TF alone we’d assign this sentence the same relevance as had we searched ‘bananas’.

That’s fine until we begin comparing documents, or searching with longer queries. We don’t want words like ‘the’,* ‘is’*, or *‘it’* to be ranked as highly as *‘bananas’* or *‘street’*.

Ideally, we want matches between rarer words to score higher. To do this, we can multiply TF by the second term — IDF. The Inverse Document Frequency measures how common a word is across all of our documents.

<img src='https://d33wubrfki0l68.cloudfront.net/8b01f75b97e0fb3bac30b88c8ad81fbcff9f2b15/1769f/images/semantic-search-11.png' width='600'/>

>The inverse document frequency (IDF) component of TF-IDF counts the number of documents that contain our query.

In this example, we have three sentences. When we calculate the IDF for our common word ‘is’, we return a much lower number than that for the rarer word ‘forest’.

If we were to then search for both words ‘is’ and ‘forest’ we would merge TF and IDF like so:

<img src='https://d33wubrfki0l68.cloudfront.net/270adb0a0e4e7e9e7b1125e4338088525d372b55/b2672/images/semantic-search-12.png' width='600'/>

We calculate the TF(‘is’, D) and TF(‘forest’, D) scores for docs a, b, and c. The IDF value is across all docs — so we calculate just IDF(‘is’) and IDF(‘forest’) once. Then, we get TF-IDF values for both words in each doc by multiplying the TF and IDF components. Sentence a scores highest for ‘forest’, and ‘is’ always scores 0 as the IDF(‘is’) score is 0.


##Setup

In [ ]:
import numpy as np

In [ ]:
def calculate_tfidf(word, sentences):
  tf = []
  num_docs = 0
  for sentence in sentences:
    # calculate TF
    term_count = len([x for x in sentence if word in sentence])
    tf.append(term_count / len(sentence))

    # count number of docs for IDF
    num_docs += 1 if word in sentence else 0
  # calculate IDF
  idf = np.log10(len(sentences) / num_docs)
  return [round(_tf * idf, 2) for _tf in tf]

##Calculate TF-IDF

In [ ]:
a = "purple is the best city in the forest".split()
b = "there is an art to getting your way and throwing bananas on to the street is not it".split()
c = "it is not often you find soggy bananas on the street".split()

In [ ]:
tfidf_a, tfidf_b, tfidf_c = calculate_tfidf("forest", [a, b, c])
print(f"TF-IDF a: {tfidf_a}\nTF-IDF b: {tfidf_b}\nTF-IDF c: {tfidf_c}")

TF-IDF a: 0.48
TF-IDF b: 0.0
TF-IDF c: 0.0


That’s great, but where does vector similarity search come into this? 

## Vectors

Well, we take our vocabulary (a big list of all words in our dataset) — and calculate the TF-IDF for each and every word.

<img src='https://d33wubrfki0l68.cloudfront.net/5fd4ff23c48442ae956dc95a6040732793440168/33a4d/images/semantic-search-13.png' width='600'/>

>We calculate the TF-IDF value for every word in our vocabulary to create a TF-IDF vector. This process is repeated for each document.

We can put all of this together to create our TF-IDF vectors like so:

In [ ]:
vocab = set(a + b + c)
print(vocab)

{'in', 'you', 'and', 'the', 'your', 'purple', 'bananas', 'often', 'art', 'it', 'an', 'there', 'forest', 'is', 'best', 'throwing', 'getting', 'to', 'way', 'street', 'on', 'not', 'find', 'soggy', 'city'}


In [ ]:
# initialize vectors
vec_a = []
vec_b = []
vec_c = []

for word in vocab:
  tfidf_a, tfidf_b, tfidf_c = calculate_tfidf(word, [a, b, c])
  vec_a.append(tfidf_a)
  vec_b.append(tfidf_b)
  vec_c.append(tfidf_c)

print(vec_a)

[0.48, 0.0, 0.0, 0.0, 0.0, 0.48, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.48, 0.0, 0.48, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.48]


In [ ]:
print(vec_b)

[0.0, 0.0, 0.48, 0.0, 0.48, 0.0, 0.18, 0.0, 0.48, 0.18, 0.48, 0.48, 0.0, 0.0, 0.0, 0.48, 0.48, 0.48, 0.48, 0.18, 0.18, 0.18, 0.0, 0.0, 0.0]


In [ ]:
print(vec_c)

[0.0, 0.48, 0.0, 0.0, 0.0, 0.0, 0.18, 0.48, 0.0, 0.18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18, 0.18, 0.18, 0.48, 0.48, 0.0]


From there we have our TF-IDF vector. It’s worth noting that vocab sizes can easily be in the 20K+ range, so the vectors produced using this method are incredibly sparse — which means we cannot encode any semantic meaning.